# **MovieLens (small) Dataset cont**
**In This notebook we show movies similarity and training Recommendation Model using preprocessed Data Data**<br>
***Our Team:***<br>
- Sara Sameh Elzahaby
- Ola Adel Hussien
- Nawal Mahmoud Shehata
- Lina Mohamed Noureldin
- Marina Magdy Ramzy<br>

ITI_ALEX_Bransh \

## `1.` Importing Libraries

In [ ]:
pip install LibRecommender

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from libreco.data import DatasetFeat, split_by_ratio_chrono
from libreco.algorithms import NCF
from libreco.evaluation import evaluate
import tensorflow as tf
import pickle
from libreco.algorithms import NCF

Instructions for updating:
non-resource variables are not supported in the long term


## `2.` Loading Dataset

In [ ]:
# mount drive to
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Read data
df = pd.read_csv('/content/drive/MyDrive/Recommender project/movies_rs (1).csv')

In [ ]:
df

,user,item,rating,time,title,year,Documentary,Children,Comedy,Romance,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,1,1,4.0,218,Toy Story,1995.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,3,4.0,218,Grumpier Old Men,1995.0,0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,6,4.0,218,Heat,1995.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,47,5.0,218,Seven (a.k.a. Se7en),1995.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,50,5.0,218,"Usual Suspects, The",1995.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102672,610,166534,4.0,16,Split,2017.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102673,610,168248,5.0,16,John Wick: Chapter Two,2017.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102674,610,168250,5.0,16,Get Out,2017.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102675,610,168252,5.0,16,Logan,2017.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `3.` Movies Recommendation

### Utility matrix

In [ ]:
# prompt: i want only to select from df_sparse user and movie and rating
Df_of_similarity=df[['user', 'item', 'rating']]
Df_of_similarity.drop_duplicates(inplace=True)
Df_of_similarity

<ipython-input-7-8b8d210fc7f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Df_of_similarity.drop_duplicates(inplace=True)


,user,item,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
102672,610,166534,4.0
102673,610,168248,5.0
102674,610,168250,5.0
102675,610,168252,5.0


In [ ]:
# Pivot the DataFrame to create a utility matrix where rows represent users,
# columns represent items, and values represent ratings or similarity scores.
utility_matrix = Df_of_similarity.pivot(index='user', columns='item', values='rating')

# Fill missing values (NaN) with 0, assuming no interaction or similarity.
utility_matrix.fillna(0, inplace=True)

# Display the first 3 rows of the utility matrix.
utility_matrix.head(3)


item,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
user,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def get_similarity_matrix(utility_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Compute the similarity matrix based on a utility matrix using cosine similarity.

    Parameters:
    utility_matrix (pd.DataFrame): A DataFrame representing the utility matrix where rows
                                   correspond to users, columns correspond to items, and
                                   values represent ratings or interactions.

    Returns:
    pd.DataFrame: A DataFrame representing the similarity matrix where rows and columns
                  correspond to items, and values represent the cosine similarity between
                  items based on user-item interactions.
    """
    # Compute cosine similarity between items (transpose of the utility matrix)
    similarity_matrix = cosine_similarity(utility_matrix.T)

    # Convert the similarity matrix array to a DataFrame with item names as index and columns
    similarity_matrix = pd.DataFrame(similarity_matrix, index=utility_matrix.columns, columns=utility_matrix.columns)

    return similarity_matrix

In [ ]:
# Example usage:
similarity_matrix= get_similarity_matrix(utility_matrix)
similarity_matrix.head()

item,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
item,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.410562,0.296917,0.035573,0.308762,0.376316,0.277491,0.131629,0.232586,0.395573,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.410562,1.000000,0.282438,0.106415,0.287795,0.297009,0.228576,0.172498,0.044835,0.417693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.296917,0.282438,1.000000,0.092406,0.417802,0.284257,0.402831,0.313434,0.304840,0.242954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.035573,0.106415,0.092406,1.000000,0.188376,0.089685,0.275035,0.158022,0.000000,0.095598,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.308762,0.287795,0.417802,0.188376,1.000000,0.298969,0.474002,0.283523,0.335058,0.218061,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
similarity_matrixf = pd.read_csv('/content/drive/MyDrive/Recommender project/similarity_matrixf.csv')

In [ ]:
def recommend_movies(movie_title: str, top_n: int, movies_df: pd.DataFrame = df, similarity_matrix: pd.DataFrame = similarity_matrixf) -> pd.DataFrame:
    """
    Recommend similar movies based on a given movie title.

    Parameters:
    movie_title (str): The title of the movie for which recommendations are sought.
    top_n (int): The number of similar movies to recommend.
    movies_df (pd.DataFrame, optional): DataFrame containing movie data with columns 'item' and 'title'.
                                        Default is set to 'df'.
    similarity_matrix (pd.DataFrame, optional): DataFrame containing the similarity scores between movies.
                                                 Default is set to 'similarity_matrixf'.

    Returns:
    pd.DataFrame: A DataFrame containing the top N similar movies to the given movie title
                  along with their similarity scores.
    """
    # Find the movie_id of the given movie_title
    movie_id = movies_df[movies_df['title'] == movie_title]['item'].values[0]

    # Get the similarity scores for all movies compared to the given movie.
    movie_id_str = str(movie_id)
    similarity_scores = similarity_matrix[movie_id_str].sort_values(ascending=False)

    # Get the top_n most similar movies based on the similarity scores.
    top_n_movies = similarity_scores.iloc[1:top_n+1]

    # Map the movie IDs to their corresponding titles.
    movie_titles = dict(zip(movies_df['item'], movies_df['title']))

    # Create a DataFrame for the top_n most similar movies and their similarity scores
    top_n_movies_df = pd.DataFrame({
        'Title': [movie_titles[movie_id] for movie_id in top_n_movies.index],
        'Similarity': top_n_movies.values
    })

    return top_n_movies_df

# Example usage:
movie_title = "Toy Story"  # Replace with the title of the movie you are interested in
top_n = 10  # Number of top similar movies you want
recommendations = recommend_movies(movie_title, top_n)
recommendations


,Title,Similarity
0,Enemy of the State,0.572601
1,Being Human,0.565637
2,Bread and Chocolate (Pane e cioccolata),0.564262
3,Don Juan DeMarco,0.557388
4,"Secret of Roan Inish, The",0.547096
5,Swimming with Sharks,0.541145
6,Some Like It Hot,0.541089
7,Super Mario Bros.,0.538913
8,"Inspector General, The",0.534169
9,Night of the Living Dead,0.530381


##`4.`User Recommendation


##`1.` Split dataset


In [ ]:
# Split Data
train_data, validation_data = split_by_ratio_chrono(df, test_size=0.2)
val_data, test_data = split_by_ratio_chrono(validation_data, test_size=0.5)

In [ ]:
train_length = len(train_data)
validation_length = len(val_data)
test_length = len(test_data)

# Print the lengths
print(f"Length of the train set: {train_length}")
print(f"Length of the validation set: {validation_length}")
print(f"Length of the test set: {test_length}")

Length of the train set: 82140
Length of the validation set: 9998
Length of the test set: 8632


## `2.` Build train and test sets

In [ ]:
train_data, data_info = DatasetFeat.build_trainset(train_data)
val_data = DatasetFeat.build_testset(val_data)
test_data = DatasetFeat.build_testset(test_data)

## `3.` Build NCF Model

In [ ]:
# Convert the string of hidden units into a list of integers
hidden_units = [int(unit) for unit in "128,64,32".split(',')]

# Create an instance of the NCF model
ncf = NCF(
    task="rating",
    data_info=data_info,
    embed_size=16,
    n_epochs=10,
    lr=0.001,
    batch_size=256,
    num_neg=4,
    use_bn=True,
    dropout_rate=None,
    hidden_units=hidden_units,
    reg=None,
)

##`4.`Train NCF Model

In [ ]:
ncf.fit(train_data, neg_sampling=False, verbose=2, eval_data=val_data, metrics=["rmse", "mae"])

Training start time: 2024-05-21 20:18:15


/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
Instructions for updating:
Colocations handled automatically by placer.
/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
train: 100%|██████████| 1611/1611 [00:06<00:00

Epoch 1 elapsed: 6.165s
	 train_loss: 0.0099


eval_pointwise: 100%|██████████| 2/2 [00:00<00:00, 16.52it/s]


	 eval rmse: 0.0005
	 eval mae: 0.0003


...

	 eval rmse: 0.0001
	 eval mae: 0.0001


train: 100%|██████████| 1611/1611 [00:04<00:00, 364.91it/s]


Epoch 10 elapsed: 4.424s
	 train_loss: 0.0


eval_pointwise: 100%|██████████| 2/2 [00:00<00:00, 59.18it/s]


	 eval rmse: 0.0002
	 eval mae: 0.0001


## `5.` Evaluate NCF Model

In [ ]:
print("Evaluation on test data:")
evaluate(ncf, test_data, neg_sampling=False, eval_batch_size=8192, metrics=["rmse", "mae"])

Evaluation on test data:


eval_pointwise: 100%|██████████| 2/2 [00:00<00:00, 80.67it/s]


{'rmse': 0.00019941166, 'mae': 0.0001079182}

##`6.` Get top 10 movie recommendations for a user

In [ ]:
# Example usage
user_id = 5  # Example user ID
top_k = 10
rec_movies = ncf.recommend_user(user=user_id, n_rec=top_k)
rec_movies[user_id]

array([  852, 89774,   490,  6104, 89492, 52973, 59900,  3421, 36525,
       47810])

In [ ]:
df[df['item'] == 1608]['title'].values[0]

'Air Force One'

In [ ]:
def get_movie_title(movie_id: int, movies_df: pd.DataFrame) -> str:
    """
    Get the title of a movie based on its ID.

    Parameters:
    movie_id (int): The ID of the movie.
    movies_df (pd.DataFrame): DataFrame containing movie data with columns 'item' and 'title'.

    Returns:
    str: The title of the movie corresponding to the given ID.
    """
    # Find the title of the movie with the given ID
    title = movies_df[movies_df['item'] == movie_id]['title'].values[0]

    return title

In [ ]:
# Print the movie titles
print("Top 10 movie recommendations for user ID {}:".format(user_id))
for movie_id in rec_movies[user_id]:
    title = get_movie_title(movie_id, df)
    print(f"Movie ID: {movie_id}, Title: {title}")

Top 10 movie recommendations for user ID 5:
Movie ID: 852, Title: Tin Cup
Movie ID: 89774, Title: Warrior
Movie ID: 490, Title: Malice
Movie ID: 6104, Title: Monty Python Live at the Hollywood Bowl
Movie ID: 89492, Title: Moneyball
Movie ID: 52973, Title: Knocked Up
Movie ID: 59900, Title: You Don't Mess with the Zohan
Movie ID: 3421, Title: Animal House
Movie ID: 36525, Title: Just Like Heaven
Movie ID: 47810, Title: Wicker Man, The


##`7.` Save NCF Model

In [ ]:
#saving model
ncf.save("ncf_user_model.h5", model_name="ncf_model")

# Save the data_info object to a file
with open('data_info.pkl', 'wb') as f:
    pickle.dump(data_info, f)

file folder ncf_user_model.h5 doesn't exists, creating a new one...


In [ ]:
def user_recommendation(user_id: int, top_k: int, model, movies_df: pd.DataFrame) -> list:
    """
    Generate movie recommendations for a given user using a recommendation model.

    Parameters:
    user_id (int): The ID of the user for whom recommendations are generated.
    top_k (int): The number of recommendations to generate.
    model: The recommendation model used to generate recommendations.
    movies_df (pd.DataFrame): DataFrame containing movie data with columns 'item' and 'title'.

    Returns:
    list: A list of tuples containing the recommended movie IDs and their titles.
    """
    # Get top k recommended movie IDs for the given user
    rec_movies = model.recommend_user(user=user_id, n_rec=top_k)

    # Define a helper function to get the movie title from the DataFrame
    def get_movie_title(movie_id, movies_df):
        title = movies_df[movies_df['item'] == movie_id]['title'].values[0]
        return title

    # Create a list to store the recommendations
    recommendations = []

    # Print the movie titles and add them to the recommendations list
    print("Top {} movie recommendations for user ID {}:".format(top_k, user_id))
    for movie_id in rec_movies[user_id]:
        title = get_movie_title(movie_id, movies_df)
        print(f"Movie ID: {movie_id}, Title: {title}")
        recommendations.append((movie_id, title))

    return recommendations


In [ ]:
# Example usage
user_id = 5  # Example user ID
top_k = 10
model = ncf  # Assuming ncf is your trained model
movies_df = df  # Assuming df is your DataFrame with movie details

# Get and print the top k recommendations
top_k_recommendations = user_recommendation(user_id, top_k, model, movies_df)

## `8.` Loading the Model and Data Info


In [ ]:
# Load the data_info object
with open('data_info.pkl', 'rb') as f:
    data_info = pickle.load(f)

In [ ]:
# Load the model
ncf = NCF.load("ncf_user_model.h5", model_name="ncf_model", data_info=data_info)